## Notebook to keep track of tabular data (and generate bagel for now)
- Files parsed:
    - `mr_proc_manifest.csv`: Get available participants and sessions 
    - `QPN_Imaging_Codes.xlsx`: Get demographics and clinical scores 
- Currently collating and tracking following assessments:
    - UPDRS
    - MoCA
- Files generated:
    - `bagel.csv`

In [4]:
import pandas as pd
import numpy as np
from itertools import product

In [2]:
release_dir = "/home/nikhil/projects/Parkinsons/qpn/releases/"

current_release = "Apr_2023"

# Current mr_proc manifest
current_mr_proc_manifest_csv = f"{release_dir}{current_release}/tabular/mr_proc_manifest.csv"

# Current PD-assessment CSV
clinical_assessment_xls = f"{release_dir}/{current_release}/tabular/QPN_Imaging_Codes.xlsx"

# NeuroBagel CSVs
clinical_scores_csv = f"{release_dir}/{current_release}/tabular/clinical_scores_summary.csv"
bagel_csv = f"{release_dir}/{current_release}/tabular/bagel.csv"


### Read mr_proc manifest


In [3]:
visit_list = ["V01"] 
session_id_list = ["01"]
session_list = [f"ses-{idx}" for idx in session_id_list]

manifest_cols = ["participant_id", "visit", "session", "bids_id"]

current_mr_proc_manifest_df = pd.read_csv(current_mr_proc_manifest_csv)
current_mr_proc_manifest_df = current_mr_proc_manifest_df[current_mr_proc_manifest_df["visit"].isin(visit_list)]
current_mr_proc_manifest_df = current_mr_proc_manifest_df[current_mr_proc_manifest_df["session"].isin(session_list)]
current_mr_proc_manifest_df = current_mr_proc_manifest_df[manifest_cols]
mr_proc_participants = current_mr_proc_manifest_df["participant_id"].unique()
n_participants = len(mr_proc_participants)
print(f"n_participants: {n_participants}")
current_mr_proc_manifest_df.head()

n_participants: 208


,participant_id,visit,session,bids_id
0,MNI0056,V01,ses-01,sub-MNI0056
1,MNI0058,V01,ses-01,sub-MNI0058
2,MNI0068,V01,ses-01,sub-MNI0068
3,MNI0079,V01,ses-01,NaN
4,MNI0103,V01,ses-01,sub-MNI0103


### Read QPN_Imaging_Codes
    - Calculate age per visit (in floats)

In [4]:
demo_cols = ["participant_id", "sex", "dob", "group", "visit_01"]
demo_df =  pd.read_excel(clinical_assessment_xls,sheet_name="recruit_manifest", engine='openpyxl').dropna(how="all", axis=0)
demo_df = demo_df[demo_cols]
demo_participants = demo_df["participant_id"].unique()
n_demo_participants = len(demo_participants)
print(f"n_demo_participants: {n_demo_participants}")

current_mr_proc_manifest_df = pd.merge(current_mr_proc_manifest_df, demo_df, on=["participant_id"], how="left")

visit_date_col = "visit_01"

# Parse and format dates: doing it in two steps because simple "format="%d-%m-%Y" skips MRI02 dates
current_mr_proc_manifest_df["dob"] = pd.to_datetime(current_mr_proc_manifest_df["dob"], errors="coerce", dayfirst=True)
current_mr_proc_manifest_df["dob"] = pd.to_datetime(current_mr_proc_manifest_df["dob"], errors="coerce", format="%d-%m-%Y")

current_mr_proc_manifest_df[visit_date_col] = pd.to_datetime(current_mr_proc_manifest_df[visit_date_col], errors="coerce", dayfirst=True)
current_mr_proc_manifest_df[visit_date_col] = pd.to_datetime(current_mr_proc_manifest_df[visit_date_col], errors="coerce", format="%d-%m-%Y")

current_mr_proc_manifest_df[f"age"] = (current_mr_proc_manifest_df[visit_date_col] - current_mr_proc_manifest_df["dob"])
current_mr_proc_manifest_df[f"age"] = np.round(current_mr_proc_manifest_df[f"age"] / np.timedelta64(1, 'Y'), 1)
 
current_mr_proc_manifest_df.head()

n_demo_participants: 208


,participant_id,visit,session,bids_id,sex,dob,group,visit_01,age
0,MNI0056,V01,ses-01,sub-MNI0056,M,1942-05-21,PD,2021-08-18,79.2
1,MNI0058,V01,ses-01,sub-MNI0058,M,1964-03-14,PD,2021-08-18,57.4
2,MNI0068,V01,ses-01,sub-MNI0068,M,1952-08-05,PD,2021-08-27,69.1
3,MNI0079,V01,ses-01,NaN,F,1971-11-25,PD,2021-12-22,50.1
4,MNI0103,V01,ses-01,sub-MNI0103,M,1939-08-17,PD,2021-08-11,82.0


### Read UPDRS and MoCA for imaging subjects

In [5]:
MoCA_df =  pd.read_excel(clinical_assessment_xls,sheet_name="MoCA", engine='openpyxl').dropna(how="all", axis=0)
MoCA_df["moca_date"] = pd.to_datetime(MoCA_df["moca_date"])
print(f"MoCA df shape: {MoCA_df.shape}")
print("Droping participants without a MoCA date")
MoCA_df = MoCA_df[~MoCA_df["moca_date"].isna()]
print(f"MoCA df shape: {MoCA_df.shape}")

participants_multiple_visits = MoCA_df[MoCA_df.duplicated(subset=["participant_id"])]["participant_id"].unique()
print(f"Number of participants with multiple visits: {len(participants_multiple_visits)}")
print("Dropping second or third visits for now - need to define visits based on the time interval between visits")
MoCA_df = MoCA_df.sort_values(by=["participant_id","moca_date"]).drop_duplicates(subset=["participant_id"],keep="first")

print(f"MoCA df shape: {MoCA_df.shape}")
MoCA_df.head()

MoCA df shape: (232, 26)
Droping participants without a MoCA date
MoCA df shape: (179, 26)
Number of participants with multiple visits: 27
Dropping second or third visits for now - need to define visits based on the time interval between visits
MoCA df shape: (151, 26)


,participant_id,delete,delete.1,moca_date,moca_vse,moca_nam,moca_att,moca_lang,moca_abs,moca_del,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,MNI0056,NaN,T1,2021-11-06,2.0,3.0,4.0,1.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MNI0058,NaN,T1,2021-07-23,4.0,3.0,5.0,2.0,0.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MNI0079,NaN,T2,2021-12-22,2.0,3.0,5.0,3.0,2.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MNI0103,NaN,T1,2021-08-11,1.0,3.0,5.0,2.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,MNI0109,NaN,T1,2021-11-24,5.0,3.0,6.0,3.0,2.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Identify duplicate participants (i.e. multiple visits)

In [6]:
UPDRS_df =  pd.read_excel(clinical_assessment_xls,sheet_name="MDS-UPDRS", engine='openpyxl').dropna(how="all", axis=0)
UPDRS_df = UPDRS_df.drop(0) # column name description gets assigned to first row for some reason

UPDRS_df["updrs_date"] = pd.to_datetime(UPDRS_df["updrs_date"])
print(f"UPDRS df shape: {UPDRS_df.shape}")
print("Droping participants without a UPDRS date")
UPDRS_df = UPDRS_df[~UPDRS_df["updrs_date"].isna()]
print(f"UPDRS df shape: {UPDRS_df.shape}")

participants_multiple_visits = UPDRS_df[UPDRS_df.duplicated(subset=["participant_id"])]["participant_id"].unique()
print(f"Number of participants with multiple visits: {len(participants_multiple_visits)}")

print("Dropping second or third visits for now - need to define visits based on the time interval between visits")
UPDRS_df = UPDRS_df.sort_values(by=["participant_id","updrs_date"]).drop_duplicates(subset=["participant_id"],keep="first")

print(f"UPDRS df shape: {UPDRS_df.shape}")

UPDRS_df.head()

UPDRS df shape: (208, 84)
Droping participants without a UPDRS date
UPDRS df shape: (94, 84)
Number of participants with multiple visits: 0
Dropping second or third visits for now - need to define visits based on the time interval between visits
UPDRS df shape: (94, 84)


,participant_id,updrs_date,updrs_staff,updrs_in_person,updrs_1_a,updrs_1_1,updrs_1_2,updrs_1_3,updrs_1_4,updrs_1_5,...,updrs_4_1,updrs_4_2,updrs_4_3,updrs_4_4,updrs_4_5,updrs_4_6,updrs_1_total,updrs_2_total,updrs_3_total,updrs_4_total
1,MNI0056,2021-11-06,Sarah Bogard,2,1,0,0,0,1,0,...,0,0,0,0,0,0,12,22,58,0
2,MNI0058,2021-07-23,Sarah Bogard,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,6,26,0
3,MNI0068,2021-08-27,Sarah Bogard,2,3,2,0,1,1,0,...,0,0,0,0,0,0,11,8,28,0
4,MNI0079,2022-01-21,Roozbeh Sattari,2,1,0,0,0,1,0,...,1,1,1,2,1,2,13,10,22,8
5,MNI0103,2021-01-10,Sarah Bogard,2,1,1,2,3,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,28,25,43,NaN


### Merge assessments into NeuroBagel format
- Manifest: [`participant_id`, `bids_id`, `session_id`, `age`, `sex`, `dob`, `group`]
- MoCA column(s): [`moca_total`]
- UPDRS column(s): [`updrs_1_total`,`updrs_2_total`,`updrs_3_total`,`updrs_4_total`]

In [7]:
manifest_cols = ["bids_id", "age", "sex", "dob", "group"]
moca_cols = ["moca_total"]
updrs_cols = ["updrs_1_total","updrs_2_total","updrs_3_total","updrs_4_total"]

participant_ids = current_mr_proc_manifest_df["participant_id"].values
a,b = zip(*list(product(participant_ids,session_id_list)))

bagel_df = pd.DataFrame()
bagel_df["participant_id"] = a
bagel_df["session_id"] = b
bagel_df["session_id"] = bagel_df["session_id"].astype(str)

bagel_df = pd.merge(bagel_df,current_mr_proc_manifest_df[["participant_id"] + manifest_cols],
                    on="participant_id",how="left")

bagel_df = pd.merge(bagel_df,MoCA_df[["participant_id"] + moca_cols],
                    on="participant_id",how="left")
                    
bagel_df = pd.merge(bagel_df,UPDRS_df[["participant_id"] + updrs_cols],
                      on="participant_id",how="left")

print(f"bagel df shape: {bagel_df.shape}")
bagel_df.head()

bagel df shape: (208, 12)


,participant_id,session_id,bids_id,age,sex,dob,group,moca_total,updrs_1_total,updrs_2_total,updrs_3_total,updrs_4_total
0,MNI0056,01,sub-MNI0056,79.2,M,1942-05-21,PD,21.0,12,22,58,0
1,MNI0058,01,sub-MNI0058,57.4,M,1964-03-14,PD,24.0,0,6,26,0
2,MNI0068,01,sub-MNI0068,69.1,M,1952-08-05,PD,NaN,11,8,28,0
3,MNI0079,01,NaN,50.1,F,1971-11-25,PD,26.0,13,10,22,8
4,MNI0103,01,sub-MNI0103,82.0,M,1939-08-17,PD,20.0,28,25,43,NaN


### Assign availability status per assessment column

In [8]:
print(f"n_mr_proc_participants: {n_participants}")

for col in moca_cols + updrs_cols:
    bagel_df[f"{col}_status"] = ~bagel_df[col].isna()
    n_available_participants = np.sum(bagel_df[f"{col}_status"])
    print(f"{col}, n_available_participants: {n_available_participants}")

print(f"bagel df shape: {bagel_df.shape}")
bagel_df.head()

n_mr_proc_participants: 208
moca_total, n_available_participants: 151
updrs_1_total, n_available_participants: 90
updrs_2_total, n_available_participants: 90
updrs_3_total, n_available_participants: 93
updrs_4_total, n_available_participants: 89
bagel df shape: (208, 17)


,participant_id,session_id,bids_id,age,sex,dob,group,moca_total,updrs_1_total,updrs_2_total,updrs_3_total,updrs_4_total,moca_total_status,updrs_1_total_status,updrs_2_total_status,updrs_3_total_status,updrs_4_total_status
0,MNI0056,01,sub-MNI0056,79.2,M,1942-05-21,PD,21.0,12,22,58,0,True,True,True,True,True
1,MNI0058,01,sub-MNI0058,57.4,M,1964-03-14,PD,24.0,0,6,26,0,True,True,True,True,True
2,MNI0068,01,sub-MNI0068,69.1,M,1952-08-05,PD,NaN,11,8,28,0,False,True,True,True,True
3,MNI0079,01,NaN,50.1,F,1971-11-25,PD,26.0,13,10,22,8,True,True,True,True,True
4,MNI0103,01,sub-MNI0103,82.0,M,1939-08-17,PD,20.0,28,25,43,NaN,True,True,True,True,False


### Save bagel_df

In [9]:
save_bagel = False

print(f"Save path: {bagel_csv}")
if save_bagel:
    bagel_df.to_csv(clinical_scores_csv, sep="\t")
    bagel_df.to_csv(bagel_csv, sep="\t")

Save path: /home/nikhil/projects/Parkinsons/qpn/releases//Apr_2023/tabular/bagel.csv


### Compare manifest and bagel

In [5]:
release_dir = "/home/nikhil/projects/Parkinsons/qpn/"

# Current mr_proc manifest
current_mr_proc_manifest_csv = f"{release_dir}/tabular/mr_proc_manifest.csv"

# NeuroBagel CSVs
bagel_csv = f"{release_dir}/tabular/bagel.csv"

manifest_df = pd.read_csv(current_mr_proc_manifest_csv)
bagel_df = pd.read_csv(bagel_csv)

In [13]:
manifest_participant_ids = set(manifest_df["participant_id"].unique())
bagel_participant_ids = set(bagel_df["participant_id"].unique())

assert manifest_participant_ids == bagel_participant_ids